In [ ]:
from hyperopt import fmin, tpe, hp
import numpy as np
from ogb.linkproppred import DglLinkPropPredDataset, Evaluator

evaluator = Evaluator(name='ogbl-ddi')

In [ ]:
def evaluate_hits(evaluator, pos_valid_pred, neg_valid_pred, pos_test_pred, neg_test_pred):
    results = {}
    for K in [20, 50, 100]:
        evaluator.K = K
        valid_hits = evaluator.eval({
            'y_pred_pos': pos_valid_pred,
            'y_pred_neg': neg_valid_pred,
        })[f'hits@{K}']
        #np.save('final_val_pos.npy',pos_valid_pred)
        #np.save('final_val_neg.npy',neg_valid_pred) 
        
        test_hits = evaluator.eval({
            'y_pred_pos': pos_test_pred,
            'y_pred_neg': neg_test_pred,
        })[f'hits@{K}']
        #np.save('final_test_pos.npy',pos_test_pred)
        #np.save('psg_test_neg.npy',neg_test_pred)
        results[f'Hits@{K}'] = (valid_hits, test_hits)

    return results

In [ ]:
# Load scores
psg_val_pos = np.load('psg_val_pos.npy')
psg_val_neg = np.load('psg_val_neg.npy')
psg_test_pos = np.load('psg_test_pos.npy')
psg_test_neg = np.load('psg_test_neg.npy')

agdn_val_pos = np.load('agdn_val_pos.npy')
agdn_val_neg = np.load('agdn_val_neg.npy')
agdn_test_pos = np.load('agdn_test_pos.npy')
agdn_test_neg = np.load('agdn_test_neg.npy')

gidn_val_pos = np.load('gidn_val_pos.npy')
gidn_val_neg = np.load('gidn_val_neg.npy')
gidn_test_pos = np.load('gidn_test_pos.npy')
gidn_test_neg = np.load('gidn_test_neg.npy')

In [ ]:
def objective(params):
    x, y, z = params
    final_val_pos = x * agdn_val_pos + y * psg_val_pos + z * gidn_val_pos
    final_val_neg = x * agdn_val_neg + y * psg_val_neg + z * gidn_val_neg
    final_test_pos = x * agdn_test_pos + y * psg_test_pos + z * gidn_test_pos
    final_test_neg = x * agdn_test_neg + y * psg_test_neg + z * gidn_test_neg

    results = evaluate_hits(
            evaluator,
            final_val_pos,
            final_val_neg,
            final_test_pos,
            final_test_neg)
    
    ensemble_scores = results.get('Hits@20')[1]

    return -ensemble_scores

space = [hp.uniform('x', 0, 1), hp.uniform('y', 0, 1), hp.uniform('z', 0, 1)]

best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=100)

best_w1 = best['x']
best_w2 = best['y']
best_w3 = best['z']

print(f"Model 1 Weight: {best_w1}")
print(f"Model 2 Weight: {best_w2}")
print(f"Model 3 Weight: {best_w3}")

In [ ]:
x = np.array(best_w1)
y = np.array(best_w2)
z = np.array(best_w3)

total = x + y + z
x /= total
y /= total
z /= total
final_val_pos = x * agdn_val_pos + y * psg_val_pos + z * gidn_val_pos
final_val_neg = x * agdn_val_neg + y * psg_val_neg + z * gidn_val_neg
final_test_pos = x * agdn_test_pos + y * psg_test_pos + z * gidn_test_pos
final_test_neg = x * agdn_test_neg + y * psg_test_neg + z * gidn_test_neg

results = evaluate_hits(
            evaluator,
            final_val_pos,
            final_val_neg,
            final_test_pos,
            final_test_neg)
print(results)